In [3]:
import os
import sys
import time
import pandas as pd
import numpy as np
# from bioservices import BioDBNet
from transcriptomic_gen import *

In [129]:
# Load Proteomics
datafilename = 'ni.3693-S5.xlsx'
data_sheet = list(range(1)) # first 5 sheets
dataFullPath = os.path.join(projectdir, 'data', datafilename)
fulldata = pd.read_excel(dataFullPath, sheet_name=data_sheet, header=0)
proteomics_data = fulldata[0]

suppfilename = 'Supplementary Data 1.xlsx'
suppFullPath = os.path.join(projectdir, 'data', suppfilename)
supp_sheet = ['Proteomics']
supplements = pd.read_excel(suppFullPath, sheet_name=supp_sheet, header=0)
Proteomics = supplements['Proteomics']

In [130]:
# Preprocess data, drop na, duplicate ';' in Gene names, 
proteomics_data['Gene names'] = proteomics_data['Gene names'].astype(str)
proteomics_data.dropna(subset=['Gene names'],inplace=True)
pluralnames = proteomics_data[proteomics_data['Gene names'].str.contains(';')==True]
for idx, row in pluralnames.iterrows():
    # print('{}: {}'.format(idx, row['Gene names']))
    names = row['Gene names'].split(';')
    rows = []
    for name in names:
        rowcopy = row.copy()
        rowcopy['Gene names'] = name
        rows.append(rowcopy)
    proteomics_data.drop(index=idx,inplace=True)
    proteomics_data.append(rows, ignore_index=True)

proteomics_data.rename(columns={'Gene names':'Gene Symbol'}, inplace=True)



In [131]:
sym2id = fetch_entrez_gene_id(proteomics_data['Gene Symbol'].tolist(), input_db='Gene Symbol')
sym2id.loc[sym2id['Gene ID']=='-', ['Gene ID']] = np.nan

proteomics_data.set_index('Gene Symbol', inplace=True)
proteomics_data['ENTREZ_GENE_ID'] = sym2id['Gene ID']
proteomics_data.dropna(subset=['ENTREZ_GENE_ID'], inplace=True)
proteomics_data.set_index('ENTREZ_GENE_ID', inplace=True)
proteomics_data.drop(columns=['Majority protein IDs'], inplace=True)

proteomics_data.to_csv(os.path.join(projectdir,'data','proteomics_data.csv'),index_label='ENGREZ_GENE_ID')
sym2id.to_csv(os.path.join(projectdir,'data','proteomics_entrez_map.csv'), index_label='Gene Symbol')

INFO    [bioservices:BioDBNet]:  Initialising BioDBNet service (REST)


retrieve 0:500
retrieve 500:1000
retrieve 1000:1500
retrieve 1500:2000
retrieve 2000:2500
retrieve 2500:3000
retrieve 3000:3500
retrieve 3500:4000
retrieve 4000:4500
retrieve 4500:5000
retrieve 5000:5500
retrieve 5500:6000
retrieve 6000:6500
retrieve 6500:7000
retrieve 7000:7500
retrieve 7500:8000
retrieve 8000:8500
retrieve 8500:9000
retrieve 9000:9500
retrieve 9500:10000
retrieve 10000:10264


In [120]:
pro_data = pd.read_csv(os.path.join(projectdir,'data','proteomics_data.csv'),index_col='ENGREZ_GENE_ID')
pro_maps = pd.read_csv(os.path.join(projectdir,'data','proteomics_entrez_map.csv'), index_col='Gene Symbol')

In [128]:
sym2id.loc[sym2id['Gene ID']=='-', ['Gene ID']] = np.nan
# sym2id.dropna(subset=['Gene ID'], inplace=True)
sym2id.to_csv(os.path.join(projectdir,'data','proteomics_entrez_map.csv'), index_label='Gene Symbol')

## Load Data by Test

In [168]:
tests = []
files = []
datas = []
for test in list(Proteomics):
    if test == 'Statistics':
        break
    cols = Proteomics[test].dropna().tolist()
    testdata = proteomics_data.loc[:,cols]
    # Logical Calculation
    thresholds = testdata.quantile(0.75, axis=0)
    testbool = pd.DataFrame(0,columns=list(testdata),index=testdata.index)
    for col in list(testdata):
        testbool.loc[testdata[col] > thresholds[col], [col]] = 1

    testdata['pos'] = (testdata > 0).sum(axis=1)/testdata.count(axis=1)
    testdata['0.5'] = 0
    testdata.loc[(testdata['pos'] > 0.5),['0.5']] = 1
    testdata['top 0.25'] = 0
    testdata.loc[testbool.any(axis=1),'top 0.25'] = 1
    
    output_path = os.path.join(projectdir,'data','Proteomics_{}.csv'.format(test.strip()))
    testdata.to_csv(output_path, index_label='ENGREZ_GENE_ID')
    print('Test Data Saved to {}'.format(output_path))
    tests.append(test.strip())
    files.append(output_path)
    datas.append(testdata)
    
protemoics_dict = dict(zip(tests,files))
testdata_dict = dict(zip(tests,datas))

Test Data Saved to /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Naïve.csv
Test Data Saved to /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th1.csv
Test Data Saved to /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th2.csv
Test Data Saved to /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th17.csv


In [165]:
# Read data from csv files
tests = []
datas = []
for test in list(Proteomics):
    if test == 'Statistics':
        break
    cols = Proteomics[test].dropna().tolist()
    testdata = proteomics_data.loc[:,cols]
    output_path = os.path.join(projectdir,'data','Proteomics_{}.csv'.format(test.strip()))
    testdata = pd.read_csv(output_path, index_col='ENGREZ_GENE_ID')
    print('Test Data Load From {}'.format(output_path))
    tests.append(test.strip())
    datas.append(testdata)
    
testdata_dict = dict(zip(tests,datas))

Test Data Load From /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Naïve.csv
Test Data Load From /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th1.csv
Test Data Load From /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th2.csv
Test Data Load From /Users/zhzhao/Dropbox/Helikar/pipelines/data/Proteomics_Th17.csv


In [143]:
testdata_bool = testdata > 0

In [166]:
thresholds = testdata.quantile(0.75, axis=0)
testbool = pd.DataFrame(0,columns=list(testdata),index=testdata.index)
for col in list(testdata):
    testbool.loc[testdata[col] > thresholds[col], [col]] = 1

testdata['pos'] = (testdata > 0).sum(axis=1)/testdata.count(axis=1)
testdata['0.5'] = 0
testdata.loc[(testdata['pos'] > 0.5),['0.5']] = 1
testdata['top 0.25'] = 0
testdata.loc[testbool.any(axis=1),'top 0.25'] = 1

In [169]:
testdata

,CopyNumber_Th17_01_steady-state,CopyNumber_Th17_02_steady-state,CopyNumber_Th17_03_steady-state,CopyNumber_Th17_04_steady-state,pos,0.5,top 0.25
ENTREZ_GENE_ID,,,,,,,
5269,0,0,0,0,0.00,0,0
1048,0,0,0,0,0.00,0,0
8828,0,0,0,0,0.00,0,0
3069,44039,36377,44926,43494,1.00,1,0
11309,0,0,0,0,0.00,0,0
1203,13160,16465,17464,21808,1.00,1,0
84337,69635,155942,95360,130960,1.00,1,1
286410,4322,2858,0,1825,0.75,1,0
1756,259,880,2078,521,1.00,1,0


In [160]:
tests

['Naïve', 'Th1', 'Th2', 'Th17']

In [84]:
pluralnames = testdata[testdata['Gene names'].str.contains(';')==True]
for idx, row in pluralnames.iterrows():
    print('{}: {}'.format(idx, row['Gene names']))
    names = row['Gene names'].split(';')
    rows = []
    for name in names:
        rowcopy = row.copy()
        rowcopy['Gene names'] = name
        rows.append(rowcopy)
    testdata.drop(index=idx,inplace=True)
    testdata.append(rows, ignore_index=True)

4: TMPIT;TMEM120A
9: KLRK1;hCG_2009644
10: ZSCAN26;ZNF187
28: IGLC2;IGLC3
36: IGKV2-30;IGKV2D-30
54: HEL-S-109;NUCB2;Nucb2;DCAF5
55: TXNRD1;GML
76: ERGIC2;RLN3
78: MINOS1;MINOS1-NBL1
81: GYPA;GPErik
91: TPSAB1;TPSB2
92: IFI27;P27
112: MPDU1;HBEBP2BPA
123: BRP44L;MPC1
133: IGLL5;IGLC1
169: CYP2D6;CYP2D7P
180: UBE2D3;UBE2D2
195: SELH;C11orf31
231: CAPN5;ASB2
253: TMSB15B;TMSB15A
267: MKL1;mkl1
278: PCDH11X;PCDH11Y
283: WASH3P;WASH2P
334: il15ra;IL15RA
378: ZFP91;ZFP91-CNTF
381: ALF;STON1
386: SULT1A4;SULT1A3
430: POLR2J;POLR2J3
454: RPSA;RPSAP58
459: KLHL13;KLHL9
532: TCRBV6S3A1N1T;TCRBV6S6A2T
533: TCRBV6S4A1;TCRB
578: LCHN;KIAA1147
586: GOLGA8N;GOLGA8O
587: MAP1LC3B;MAP1LC3B2
632: INS;INS-IGF2
643: SNRPB;SNRPN
652: BICD1;GLYAT
654: SNRPG;SNRPGP15
663: MIF4GD;MEOX1
680: PRR5-ARHGAP8;LOC553158;ARHGAP8
703: HEMK2;N6AMT1
704: TRIM6-TRIM34;TRIM34
709: EIF3H;EIF3S3
714: FLJ13984;METTL8
733: MED29;IXL
744: MST4;RBM4B
770: ppm1m;PPM1M
781: RANBP3;ZNF8
783: GPR89A;GPR89B;GPR89C
795: CDC2L1;CDK11

In [85]:
testdata

,CopyNumber_Th17_01_steady-state,CopyNumber_Th17_02_steady-state,CopyNumber_Th17_03_steady-state,CopyNumber_Th17_04_steady-state,Majority protein IDs,Gene names
0,0,0,0,0,A0A024QZX5;A0A087X1N8,SERPINB6
1,0,0,0,0,P06731;A0A087WYX0;A0A024R0K5;P06731-2;A0A087X0...,CEACAM5
2,0,0,0,0,O60462-5;A0A024R412;O60462-4;O60462;A0A024R3W6...,NRP2
3,44039,36377,44926,43494,A0A024R4E5;Q00341;Q00341-2;H0Y394,HDLBP
5,0,0,0,0,O94956;A0A0A0MTF1;E9PRW4;A0A024R5I4;O94956-2;O...,SLCO2B1
6,13160,16465,17464,21808,O75503;A0A024R644;A0A087WZY0,CLN5
7,69635,155942,95360,130960,P60002;K7EN05;K7EPC8;A0A024R7E8;K7EQ44;K7EMV4,ELOF1
8,10697,9469,12886,12332,A0A024RAC6;Q14241,TCEB3
11,4322,2858,0,1825,A0A067XG54;Q8NB49-3;Q8NB49;Q8NB49-2;Q8NB49-4,ATP11C
12,259,880,2078,521,P11532;A0A087WV90;E9PDN5;A0A087WTU7;A0A075B6G3...,DMD


In [105]:
testdata['Gene Symbol'].astype(str)

0        SERPINB6
1         CEACAM5
2            NRP2
3           HDLBP
5         SLCO2B1
           ...   
10514     C1orf43
10515      USP6NL
10516       SAP18
10517        TCF3
10518        ERC1
Name: Gene Symbol, Length: 10083, dtype: object